# BÀI TẬP PYTHON CƠ BẢN

## Đề Bài

Viết hàm tính tiền bảo hiểm và thuế thu nhập cá nhân cho các nhân viên:
- Input: lương, phụ cấp
- Output: BH, PIT

## Script

### Using

In [1]:
import sys
from enum import Enum
from collections import defaultdict

### Utilities

In [2]:
# Ensure that the input is always an integer
def get_input_int(cap='Nhập số nguyên: '):
    while True:
        try:
            return int(input(cap))
        except:
            print('Dữ liệu nhập vào phải là số nguyên, xin vui lòng thử lại...')

In [3]:
# Ensure that the input is always an number
def get_input_num(cap='Nhập số: '):
    while True:
        try:
            return float(input(cap))
        except:
            print('Dữ liệu nhập vào phải là số, xin vui lòng thử lại...')

In [4]:
# Ensure that the input is always a positive integer
def get_positive_integer(cap='Nhập số nguyên dương: '):
    while True:
        num = get_input_int(cap)
        if num > 0:
            return num
        print('Dữ liệu nhập vào phải là số nguyên dương, xin vui lòng thử lại...')

In [5]:
# Ensure that the input is always a non-negative number
def get_non_negative_num(cap='Nhập số không âm: '):
    while True:
        num = get_input_num(cap)
        if num >= 0:
            return num
        print('Dữ liệu nhập vào phải là số không âm, xin vui lòng thử lại...')

### Fields

In [6]:
L_CS = 1800000  # lương cơ sở 2023 (LCS)
L_TTV = 4680000  # luong tối thiểu vùng 2023 (LTTV)

In [7]:
PT_BHXH = 8  # phần trăm bảo hiểm xã hội (BHXH)
PT_BHYT = 1.5  # phần trăm bảo hiểm y tế (BHYT)
PT_BHTN = 1  # phần trăm bảo hiểm thất nghiệp (BHTN)

In [8]:
HS_BH_MAX = 20  # hệ số bảo hiểm (BH) tối đa

In [9]:
GT_TTNCN = 11000000  # giảm trừ thuế thu nhập cá nhân (TTNCN)

In [10]:
class BacThue(Enum):
    Bac0 = 0
    Bac1 = 1
    Bac2 = 2
    Bac3 = 3
    Bac4 = 4
    Bac5 = 5
    Bac6 = 6
    Bac7 = 7

In [11]:
class NhanVien:
    # Initialization
    def __init__(self, lcb=0, pc=0):
        self.lcb = lcb
        self.pc = pc

    # Set lương thực nhận
    def set_ltn(self, ltn=0):
        self.ltn = ltn

In [12]:
IDX_BT_LOW = 0  # index bậc dưới
IDX_BT_HIGH = 1  # index bậc trên
IDX_BT_NAME = 2  # index bậc thuế

# lương từ | đến dưới | bậc thuế
BTS = [[0, 5000000, BacThue.Bac1],
       [5000000, 10000000, BacThue.Bac2],
       [10000000, 18000000, BacThue.Bac3],
       [18000000, 32000000, BacThue.Bac4],
       [32000000, 52000000, BacThue.Bac5],
       [52000000, 80000000, BacThue.Bac6],
       [80000000, sys.maxsize, BacThue.Bac7]]

In [13]:
IDX_TS = 0  # index thuế suất của từng bậc thuế
IDX_HSG = 1  # index hệ số giảm của từng bậc thuế

BT_DICTS = defaultdict(list)
# {bậc thuế} - {thuế suất (%) | hệ số giảm}
BT_DICTS[BacThue.Bac0] = [0, 0]
BT_DICTS[BacThue.Bac1] = [5, 0]
BT_DICTS[BacThue.Bac2] = [10, 250000]
BT_DICTS[BacThue.Bac3] = [15, 750000]
BT_DICTS[BacThue.Bac4] = [20, 1650000]
BT_DICTS[BacThue.Bac5] = [25, 3250000]
BT_DICTS[BacThue.Bac6] = [30, 5850000]
BT_DICTS[BacThue.Bac7] = [35, 9850000]

### Common

In [14]:
# Tính bảo hiểm xã hội
calculate_bhxh = lambda lcb=0: min(lcb, L_CS * HS_BH_MAX) * PT_BHXH / 100  # mức lương đóng BHXH tối đa bằng 20 lần mức LCS

In [15]:
# Tính bảo hiểm y tế
calculate_bhyt = lambda lcb=0: min(lcb, L_CS * HS_BH_MAX) * PT_BHYT / 100 # mức lương đóng BHYT tối đa bằng 20 lần mức LCS

In [16]:
# Tính bảo hiểm thất nghiệp
calculate_bhtn = lambda lcb=0: min(lcb, L_TTV * HS_BH_MAX) * PT_BHTN / 100  # mức lương đóng BHTN tối đa bằng 20 lần mức LTTV

In [17]:
# Tính bảo hiểm
calculate_bh = lambda lcb=0: calculate_bhxh(lcb) + calculate_bhyt(lcb) + calculate_bhtn(lcb)  # gói đóng BH gồm có BHXH, BHYT & BHTN

In [18]:
# Tính lương NET
calculate_lnet = lambda lcb=0: lcb - calculate_bh(lcb)  # lương NET là lương cơ bản (LCB) sau khi đóng BH

In [19]:
# Tính lương thực tế
calculate_ltt = lambda lcb=0, pc=0: calculate_lnet(lcb) + pc  # lương thực tế là lương NET được tính thêm các khoản phụ cấp

In [20]:
# Tra bậc thuế
def lookup_bt(ltt=0):
    if ltt > GT_TTNCN:
        tntt = ltt - GT_TTNCN  # thu nhập tính thuế (TNTT) là số dư của lương thực tế (LTT) so với mức giảm trừ TTNCN
        for i in range(len(BTS)):
            bacThue = BTS[i]
            if bacThue[IDX_BT_LOW] <= tntt < bacThue[IDX_BT_HIGH]:
                return bacThue[IDX_BT_NAME]
    else:
        return BacThue.Bac0

In [21]:
# Tính thuế thu nhập cá nhân
def calculate_ttncn(ltt=0):
    btDic = BT_DICTS[lookup_bt(ltt)]
    return (ltt - GT_TTNCN) * btDic[IDX_TS] / 100 - btDic[IDX_HSG]  # TTNCN chỉ đánh trên (TNTT)

In [22]:
# Tính lương thực nhận
calculate_ltn = lambda ltt=0: ltt - calculate_ttncn(ltt)  # lương sau thuế (LST) bằng lương thực nhận (LTN)

In [25]:
# Main
def main():
    n = get_positive_integer('Nhập số lượng nhân viên: ')
    nhanViens = []
    # initialize each element in the range
    for i in range(n):
        lcb = get_non_negative_num(f'Nhập lương cơ bản cho nhân viên {i + 1}: ')
        pc = get_non_negative_num(f'Nhập phụ cấp cho nhân viên {i + 1}: ')
        nhanVien = NhanVien(lcb, pc)
        nhanViens.append(nhanVien)
        nhanVien.set_ltn(calculate_ltn(calculate_ltt(lcb, pc)))
    # calculate salary for each element in the range
    print('\nKết quả')
    for i in range(len(nhanViens)):
        print(f'Lương thực nhận của nhân viên {i + 1} là: {"{:,}".format(nhanViens[i].ltn)}')

In [26]:
# Run
main()

Nhập số lượng nhân viên: a
Dữ liệu nhập vào phải là số nguyên, xin vui lòng thử lại...
Nhập số lượng nhân viên: 1.2
Dữ liệu nhập vào phải là số nguyên, xin vui lòng thử lại...
Nhập số lượng nhân viên: -3
Dữ liệu nhập vào phải là số nguyên dương, xin vui lòng thử lại...
Nhập số lượng nhân viên: 0
Dữ liệu nhập vào phải là số nguyên dương, xin vui lòng thử lại...
Nhập số lượng nhân viên: 4
Nhập lương cơ bản cho nhân viên 1: b
Dữ liệu nhập vào phải là số, xin vui lòng thử lại...
Nhập lương cơ bản cho nhân viên 1: -5
Dữ liệu nhập vào phải là số không âm, xin vui lòng thử lại...
Nhập lương cơ bản cho nhân viên 1: 0
Nhập phụ cấp cho nhân viên 1: 44444444.4
Nhập lương cơ bản cho nhân viên 2: 88888888.8
Nhập phụ cấp cho nhân viên 2: 0
Nhập lương cơ bản cho nhân viên 3: 5000000
Nhập phụ cấp cho nhân viên 3: 17000000
Nhập lương cơ bản cho nhân viên 4: 20000000
Nhập phụ cấp cho nhân viên 4: 2000000

Kết quả
Lương thực nhận của nhân viên 1 là: 39,333,333.3
Lương thực nhận của nhân viên 2 là: 68,355